In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
%cd ~/Retrieval-Augmented-Time-Series-Forecasting

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use only GPU


In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
data= pd.read_csv("filepath")
data

,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,S12B,...,S12B_T,S12C_T,S12D_T,S18C,NP205_rain,NP205_stage,NP205_PET,P33_rain,P33_stage,P33_PET
0,2020-10-16,242.17,279.69,218.42,218.42,445.77,481.38,0.0,460,382.0,...,10.03,10.06,10.14,289.0,2.28,6.925,2.090,0.106,7.133,2.090
1,2020-10-17,296.05,294.85,265.13,265.13,587.35,556.24,0.0,490,405.0,...,10.17,10.17,10.18,274.0,0.01,7.025,3.870,0.000,7.133,3.870
2,2020-10-18,277.85,294.24,264.23,264.23,514.45,503.14,0.0,472,394.0,...,10.13,10.13,10.15,252.0,0.00,7.008,3.510,0.088,7.138,3.510
3,2020-10-19,224.13,294.72,264.39,264.39,491.06,537.91,0.0,466,392.0,...,10.12,10.12,10.14,227.0,0.11,6.986,1.980,0.256,7.167,1.980
4,2020-10-20,260.51,295.06,263.38,263.38,489.25,558.77,0.0,472,399.0,...,10.13,10.14,10.17,276.0,0.37,6.977,1.270,0.678,7.195,1.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533,2024-12-27,76.07,149.85,0.00,0.00,0.00,317.51,0.0,0,0.0,...,8.77,9.56,9.54,-28.4,0.01,6.642,1.270,0.000,7.239,1.270
1534,2024-12-28,76.09,149.96,0.00,0.00,0.00,316.96,0.0,0,0.0,...,8.76,9.54,9.53,60.6,0.02,6.639,1.524,0.100,7.235,1.524
1535,2024-12-29,228.31,97.82,0.00,0.00,0.00,474.71,0.0,0,0.0,...,8.85,9.63,9.61,269.0,1.84,6.755,1.016,1.750,7.294,1.016
1536,2024-12-30,188.28,149.24,9.90,9.90,0.00,535.99,0.0,0,0.0,...,8.84,9.62,9.60,302.0,0.00,6.800,2.032,0.000,7.327,2.032


In [ ]:
# Convert 'date' column to proper datetime format
data['date'] = pd.to_datetime(data['date'])

print(data.info())

data['formatted_date'] = data['date'].dt.strftime('%m/%d/%Y')

print(data[['date', 'formatted_date']])                                                                                                    # Drop the 'formatted_date' column if not needed
data = data.drop(columns=['formatted_date'])

data['date'] = data['date'].dt.strftime('%m/%d/%Y')

data 



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1538 entries, 0 to 1537
Data columns (total 39 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              1538 non-null   datetime64[ns]
 1   S199              1538 non-null   float64       
 2   S200              1538 non-null   float64       
 3   S332B             1538 non-null   float64       
 4   S332BN            1538 non-null   float64       
 5   S332C             1538 non-null   float64       
 6   S332D             1538 non-null   float64       
 7   S332DX1           1538 non-null   float64       
 8   S12A              1538 non-null   int64         
 9   S12B              1538 non-null   float64       
 10  S12C              1538 non-null   int64         
 11  S12D              1538 non-null   int64         
 12  S333              1538 non-null   float64       
 13  S333N             1538 non-null   float64       
 14  S334              1538 n

,date,S199,S200,S332B,S332BN,S332C,S332D,S332DX1,S12A,S12B,...,S12B_T,S12C_T,S12D_T,S18C,NP205_rain,NP205_stage,NP205_PET,P33_rain,P33_stage,P33_PET
0,10/16/2020,242.17,279.69,218.42,218.42,445.77,481.38,0.0,460,382.0,...,10.03,10.06,10.14,289.0,2.28,6.925,2.090,0.106,7.133,2.090
1,10/17/2020,296.05,294.85,265.13,265.13,587.35,556.24,0.0,490,405.0,...,10.17,10.17,10.18,274.0,0.01,7.025,3.870,0.000,7.133,3.870
2,10/18/2020,277.85,294.24,264.23,264.23,514.45,503.14,0.0,472,394.0,...,10.13,10.13,10.15,252.0,0.00,7.008,3.510,0.088,7.138,3.510
3,10/19/2020,224.13,294.72,264.39,264.39,491.06,537.91,0.0,466,392.0,...,10.12,10.12,10.14,227.0,0.11,6.986,1.980,0.256,7.167,1.980
4,10/20/2020,260.51,295.06,263.38,263.38,489.25,558.77,0.0,472,399.0,...,10.13,10.14,10.17,276.0,0.37,6.977,1.270,0.678,7.195,1.270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533,12/27/2024,76.07,149.85,0.00,0.00,0.00,317.51,0.0,0,0.0,...,8.77,9.56,9.54,-28.4,0.01,6.642,1.270,0.000,7.239,1.270
1534,12/28/2024,76.09,149.96,0.00,0.00,0.00,316.96,0.0,0,0.0,...,8.76,9.54,9.53,60.6,0.02,6.639,1.524,0.100,7.235,1.524
1535,12/29/2024,228.31,97.82,0.00,0.00,0.00,474.71,0.0,0,0.0,...,8.85,9.63,9.61,269.0,1.84,6.755,1.016,1.750,7.294,1.016
1536,12/30/2024,188.28,149.24,9.90,9.90,0.00,535.99,0.0,0,0.0,...,8.84,9.62,9.60,302.0,0.00,6.800,2.032,0.000,7.327,2.032


In [ ]:
# Replace with your actual DataFrame
df = pd.DataFrame(data)

#Convert 'date' to datetime format if not 
df['date'] = pd.to_datetime(df['date'])

#Melt the DataFrame to long format
long_df = pd.melt(df, id_vars=['date'], 
                  value_vars=[col for col in df.columns if col != 'date'], 
                  var_name='unique_id', 
                  value_name='y')

#Rename the date column to 'ds'
long_df.rename(columns={'date': 'ds'}, inplace=True)              

# Reorder columns
long_df = long_df[['unique_id', 'ds', 'y']]

long_df

# Display the resulting long format DataFrame
print(long_df) 

Y_df = long_df[['unique_id', 'ds', 'y']]                                                                                                                                                     # Rename the 'y' column to 'target'
Y_df = Y_df.rename(columns={'y': 'target'})
Y_df

      unique_id         ds        y
0          S199 2020-10-16  242.170
1          S199 2020-10-17  296.050
2          S199 2020-10-18  277.850
3          S199 2020-10-19  224.130
4          S199 2020-10-20  260.510
...         ...        ...      ...
58439   P33_PET 2024-12-27    1.270
58440   P33_PET 2024-12-28    1.524
58441   P33_PET 2024-12-29    1.016
58442   P33_PET 2024-12-30    2.032
58443   P33_PET 2024-12-31    2.794

[58444 rows x 3 columns]


,unique_id,ds,target
0,S199,2020-10-16,242.170
1,S199,2020-10-17,296.050
2,S199,2020-10-18,277.850
3,S199,2020-10-19,224.130
4,S199,2020-10-20,260.510
...,...,...,...
58439,P33_PET,2024-12-27,1.270
58440,P33_PET,2024-12-28,1.524
58441,P33_PET,2024-12-29,1.016
58442,P33_PET,2024-12-30,2.032


In [8]:
# 2. Build per-series arrays
series_map = {}
dates_map  = {}

for uid, grp in Y_df.groupby('unique_id'):
    vals = grp['target'].to_numpy(dtype=np.float32)
    ds_list = grp['ds'].tolist()
    series_map[uid] = vals
    dates_map[uid]  = ds_list

print(f"Prepared {len(series_map)} series.")


Prepared 38 series.


In [1]:
#K=4 7 days SIMRAF

In [ ]:
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import json
from pathlib import Path
import time
import logging

# ----------------------------------------------------------------------
# Logging
# ----------------------------------------------------------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger("RetrievalAug")

# ----------------------------------------------------------------------
# CONFIG
# ----------------------------------------------------------------------
prediction_length = 7
input  = 100                     #History length
top_n  = 4                       # 0 = pure baseline 
aggregate_mode = "mean"          
chronos_model_id = "amazon/chronos-bolt-small"

eval_start = pd.Timestamp("2024-05-15")
eval_end   = pd.Timestamp("2024-12-31")
eval_uids  = ["NP205_stage","P33_stage","G620_water_level","NESRS1","NESRS2"]

min_history_points = input

# overlap filter
OVERLAP_THRESH = 0.70            # >= 70% overlap -> drop duplicate window from same uid

H = input
P = prediction_length
AUG_LEN = 2 * H + P              # used only for retrieval variants

print(f"[CONFIG] H={H}, P={P}, top_n={top_n}, aggregate={aggregate_mode}, "
      f"AUG_LEN={AUG_LEN}, overlap_thr={OVERLAP_THRESH}")


# ----------------------------------------------------------------------
# REQUIRED EXTERNAL OBJECTS (series_map, dates_map) must exist
# ----------------------------------------------------------------------
from chronos_bolt_adapter import ChronosBoltAdapter

device_map = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = (torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.is_bf16_supported())
               else torch.float32)

chronos = ChronosBoltAdapter(
    model_id=chronos_model_id,
    device_map=device_map,
    torch_dtype=torch_dtype,
    faux_num_samples=20
)

eval_uids = [u for u in eval_uids if u in series_map]

# ----------------------------------------------------------------------
# Build date index map
# ----------------------------------------------------------------------
def build_date_index(dates_map):
    out = {}
    for uid, dl in dates_map.items():
        dlist = [pd.Timestamp(d) for d in dl]
        if not pd.Index(dlist).is_monotonic_increasing:
            order = np.argsort(dlist)
            dlist = [dlist[i] for i in order]
        idx_map = {d: i for i, d in enumerate(dlist)}
        out[uid] = (dlist, idx_map)
    return out

DATE_INDEX = build_date_index(dates_map)

def date_to_cutoff(uid, ds):
    ds = pd.Timestamp(ds)
    if uid not in DATE_INDEX:
        return None
    _, im = DATE_INDEX[uid]
    return im.get(ds, None)

station_mean_std = {
    k: (float(np.mean(v)), float(np.std(v) + 1e-7))
    for k, v in series_map.items()
}

def denormalize_predictions(pred, mean_std):
    mean, std = mean_std
    return pred * std + mean

# ----------------------------------------------------------------------
# BUILD CANDIDATE BANK  (H history + P future)
# ----------------------------------------------------------------------
cand_full_segments = []
cand_series        = []
cand_end_idx       = []
cand_start_idx     = []

t_build = time.time()
for uid, values in series_map.items():
    x = np.asarray(values, dtype=float)
    if len(x) < H + P:
        continue

    dates_uid = [pd.Timestamp(d) for d in dates_map[uid]]
    stride = 1
    for end in range(H, len(x) - P + 1, stride):
        end_date = dates_uid[end - 1]
        if end_date >= eval_start:  
            continue

        start = end - H
        hist_seg = x[start:end]
        fut_seg  = x[end:end+P]

        if np.isnan(hist_seg).all() or np.isnan(fut_seg).all():
            continue

        hist_seg = pd.Series(hist_seg).ffill().bfill().values
        fut_seg  = pd.Series(fut_seg).ffill().bfill().values

        full_seg = np.concatenate([hist_seg, fut_seg]) 
        cand_full_segments.append(full_seg.astype(np.float32))
        cand_series.append(uid)
        cand_end_idx.append(end)
        cand_start_idx.append(start)

cand_full_segments = (np.stack(cand_full_segments)
                      if cand_full_segments else np.zeros((0, H+P), dtype=np.float32))
cand_series    = np.array(cand_series)
cand_end_idx   = np.array(cand_end_idx)
cand_start_idx = np.array(cand_start_idx)

print(f"[BUILD] Candidate segments (history+future): {cand_full_segments.shape} "
      f"built in {time.time() - t_build:.2f}s")

uid_to_cand_indices = {}
for i, s in enumerate(cand_series):
    uid_to_cand_indices.setdefault(s, []).append(i)

cand_tensor_full = torch.from_numpy(cand_full_segments)     
cand_hist_tensor = cand_tensor_full[:, :H]                
cand_hist_mean   = cand_hist_tensor.mean(dim=1)
cand_hist_std    = cand_hist_tensor.std(dim=1) + 1e-6

# ----------------------------------------------------------------------
# Overlap helper
# ----------------------------------------------------------------------
def window_overlap_ratio(a_start, a_end, b_start, b_end):
    inter = max(0, min(a_end, b_end) - max(a_start, b_start))
    length = max(a_end - a_start, 1)
    return inter / length

# ----------------------------------------------------------------------
# RETRIEVAL (Euclidean)
# ----------------------------------------------------------------------
def retrieve_topn(query_series, query_uid, cutoff_idx, top_n):
    if top_n == 0:
        return np.array([], dtype=int), np.array([], dtype=float), query_series[cutoff_idx - H: cutoff_idx]

    if cutoff_idx is None or cutoff_idx < H or cutoff_idx + P > len(query_series):
        return None

    ctx = query_series[cutoff_idx - H: cutoff_idx]
    ctx = pd.Series(ctx).ffill().bfill().values

    q = torch.from_numpy(ctx.astype(np.float32)).unsqueeze(0)  
    with torch.no_grad():
        dists = torch.cdist(q, cand_hist_tensor, p=2).squeeze(0)

    for idx in uid_to_cand_indices.get(query_uid, []):
        if cand_end_idx[idx] >= (cutoff_idx - 1):
            dists[idx] = float("inf")

    valid = (~torch.isinf(dists)).nonzero(as_tuple=True)[0]
    if valid.numel() == 0:
        return None

    d_valid = dists[valid]
    order = torch.argsort(d_valid, descending=False)

    chosen, d_sel = [], []
    for pos in order:
        if len(chosen) >= top_n:
            break
        ci = int(valid[pos])
        keep = True
        for t in chosen:
            if cand_series[ci] == cand_series[t]:
                ov = window_overlap_ratio(
                    cand_start_idx[ci], cand_end_idx[ci],
                    cand_start_idx[t],   cand_end_idx[t]
                )
                if ov >= OVERLAP_THRESH:
                    keep = False
                    break
        if keep:
            chosen.append(ci)
            d_sel.append(float(d_valid[pos]))

    return np.array(chosen, dtype=int), np.array(d_sel, dtype=float), ctx

# ----------------------------------------------------------------------
# VARIANT BUILDERS
# ----------------------------------------------------------------------
def build_baseline_variant(query_ctx):
    t = torch.tensor(query_ctx, dtype=torch.float32)
    m = float(t.mean())
    s = float(t.std(unbiased=False) + 1e-7)
    ctx_n = (t - m) / s
    return [ctx_n], [(m, s)]

def build_augmented_variants(query_ctx, top_indices):
    ctx_t = torch.tensor(query_ctx, dtype=torch.float32)
    m_c = float(ctx_t.mean())
    s_c = float(ctx_t.std(unbiased=False) + 1e-7)
    ctx_n = (ctx_t - m_c) / s_c
    variants, stats = [], []
    zero_prefix = torch.zeros(H + P, dtype=torch.float32)
    variants.append(torch.cat([zero_prefix, ctx_n]))
    stats.append((m_c, s_c))

    for ci in top_indices:
        seg_full = cand_full_segments[ci]
        seg_t = torch.tensor(seg_full, dtype=torch.float32)
        m_s = float(seg_t.mean())
        s_s = float(seg_t.std(unbiased=False) + 1e-7)
        seg_n = (seg_t - m_s) / s_s
        shift = ctx_n[0] - seg_n[-1]
        seg_n = seg_n + shift

        variants.append(torch.cat([seg_n, ctx_n]))
        stats.append((m_c, s_c))

    for v in variants:
        assert v.shape[0] == AUG_LEN, f"Variant length mismatch {v.shape[0]} != {AUG_LEN}"

    return variants, stats

# ----------------------------------------------------------------------
# FORECAST LOOP
# ----------------------------------------------------------------------
retrieval_audit = []
records = []
eval_dates = pd.date_range(eval_start, eval_end, freq="D")

t_loop = time.time()
for ds in tqdm(eval_dates, desc="Euclidean Retrieval/Baseline"):
    for uid in eval_uids:
        cutoff = date_to_cutoff(uid, ds)
        if cutoff is None:
            continue

        series_vals = np.asarray(series_map[uid], dtype=float)
        if cutoff + P > len(series_vals) or cutoff < min_history_points:
            continue

        # baseline 
        if top_n == 0:
            query_ctx = pd.Series(series_vals[cutoff - H: cutoff]).ffill().bfill().values
            variants, stats = build_baseline_variant(query_ctx)
            baseline_mode = True
            top_idx, top_dists = np.array([],int), np.array([],float)
        else:
            ret = retrieve_topn(series_vals, uid, cutoff, top_n)
            if ret is None:
                continue
            top_idx, top_dists, query_ctx = ret
            variants, stats = build_augmented_variants(query_ctx, top_idx)
            baseline_mode = False

        # model call
        with torch.no_grad():
            samples = chronos.predict(variants, prediction_length=P, num_samples=1)

        fc_matrix = samples[:, 0, :].cpu().numpy()

        fc_matrix_denorm = np.array([
            fc_matrix[i] * stats[i][1] + stats[i][0]
            for i in range(len(fc_matrix))
        ])
        if baseline_mode:
            final_fc = fc_matrix_denorm[0]
        else:
            if aggregate_mode == "mean":
                final_fc = fc_matrix_denorm.mean(axis=0)
            else:
                anchor = np.median(top_dists) if top_dists.size>0 else 0
                d_all = np.concatenate([[anchor], top_dists])
                w = np.exp(-alpha * d_all); w /= w.sum()
                final_fc = (fc_matrix_denorm * w[:,None]).sum(axis=0)

        # Truth + record
        truth_future = series_vals[cutoff:cutoff + P]
        for h in range(P):
            records.append({
                "unique_id": uid,
                "ds": ds,
                "horizon": h+1,
                "pred": float(final_fc[h]),
                "truth": float(truth_future[h]),
            })

print(f"[LOOP] Completed in {time.time() - t_loop:.2f}s; records: {len(records)}")

# ----------------------------------------------------------------------
# RAW PREDICTIONS CSV
# ----------------------------------------------------------------------
# CSV  write
raw_preds_path = Path(f"KSTUDY_raw_preds_top{top_n}_{prediction_length}_days_Sim.csv")

df_aug = pd.DataFrame(records)
if df_aug.empty:
    raise RuntimeError("No augmented rows. Adjust params / dates / data coverage.")

# write raw preds & truth to a single CSV
df_aug.to_csv(raw_preds_path, index=False)
logger.info(f"Wrote raw predictions to {raw_preds_path}")

# ----------------------------------------------------------------------
# METRICS 
# ----------------------------------------------------------------------

df_aug = pd.DataFrame(records)
if df_aug.empty:
    raise RuntimeError("No augmented rows. Adjust params / dates / data coverage.")

# overall
err_all = df_aug.pred - df_aug.truth
overall_mae  = float(np.mean(np.abs(err_all)))
overall_rmse = float(np.sqrt(np.mean(err_all**2)))

print("=== Euclidean Retrieval ===")

# per-station
def station_metrics(group):
    err = group.pred - group.truth
    mae  = float(np.mean(np.abs(err)))
    rmse = float(np.sqrt(np.mean(err**2)))
    return pd.Series({"MAE": mae, "RMSE": rmse})

per_station = (
    df_aug
      .groupby("unique_id")
      .apply(station_metrics)
      .reset_index()
      .rename(columns={"unique_id": "station"})
)

print("Per-station metrics:")
print(per_station.to_string(index=False))

# average of station-level MAE/RMSE
avg_mae  = per_station.MAE.mean()
avg_rmse = per_station.RMSE.mean()

print(f"\nAVERAGE ACROSS STATIONS: MAE={avg_mae:.6f} | RMSE={avg_rmse:.6f}")


In [2]:
#K=4 7 days MIRAF

In [ ]:
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import json
from pathlib import Path
import time
import logging
from sklearn.feature_selection import mutual_info_regression

# ======================================================================
# Logging
# ======================================================================
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
logger = logging.getLogger("RetrievalAug_MI")

# ======================================================================
# CONFIG
# ======================================================================
prediction_length = 7
H = 100                      # history length 
top_n = 4                    # number of retrieved neighbors (0 = pure baseline)
aggregate_mode = "mean"      
chronos_model_id = "amazon/chronos-bolt-small"

eval_start = pd.Timestamp("2024-05-15")
eval_end   = pd.Timestamp("2024-12-31")
eval_uids  = ["NP205_stage","P33_stage","G620_water_level","NESRS1","NESRS2"]
min_history_points = H

OVERLAP_THRESH = 0.70        # >= 70% overlap => treat as duplicate

P = prediction_length
AUG_LEN = 2 * H + P
print(f"[CONFIG] H={H}, P={P}, top_n={top_n}, aggregate={aggregate_mode}, "
      f"AUG_LEN={AUG_LEN} (retrieval mode only), overlap_thr={OVERLAP_THRESH}")

# ======================================================================
# REQUIRED EXTERNAL OBJECTS (series_map, dates_map)
# ======================================================================
from chronos_bolt_adapter import ChronosBoltAdapter  

device_map = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = (torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.is_bf16_supported())
               else torch.float32)

chronos = ChronosBoltAdapter(
    model_id=chronos_model_id,
    device_map=device_map,
    torch_dtype=torch_dtype,
    faux_num_samples=20  
)

eval_uids = [u for u in eval_uids if u in series_map]

# ======================================================================
# Date index map
# ======================================================================
def build_date_index(dates_map):
    out = {}
    for uid, dl in dates_map.items():
        dlist = [pd.Timestamp(d) for d in dl]
        if not pd.Index(dlist).is_monotonic_increasing:
            order = np.argsort(dlist)
            dlist = [dlist[i] for i in order]
        idx_map = {d: i for i, d in enumerate(dlist)}
        out[uid] = (dlist, idx_map)
    return out

DATE_INDEX = build_date_index(dates_map)

def date_to_cutoff(uid, ds):
    ds = pd.Timestamp(ds)
    if uid not in DATE_INDEX:
        return None
    _, im = DATE_INDEX[uid]
    return im.get(ds, None)

station_mean_std = {
    k: (float(np.mean(v)), float(np.std(v) + 1e-7))
    for k, v in series_map.items()
}

def denormalize_predictions(pred, mean_std):
    mean, std = mean_std
    return pred * std + mean

# ======================================================================
# Build candidate bank (history + future segments)
# ======================================================================
cand_full_segments = []
cand_series = []
cand_end_idx = []
cand_start_idx = []

t_build = time.time()
for uid, values in series_map.items():
    x = np.asarray(values, dtype=float)
    if len(x) < H + P:
        continue
    dates_uid = [pd.Timestamp(d) for d in dates_map[uid]]
    stride = H 
    for end in range(H, len(x) - P + 1, stride):
        end_date = dates_uid[end - 1]
        if end_date >= eval_start:
            continue 
        start = end - H
        hist_seg = x[start:end]
        fut_seg  = x[end:end+P]
        if np.isnan(hist_seg).all() or np.isnan(fut_seg).all():
            continue
        if np.isnan(hist_seg).any():
            hist_seg = (pd.Series(hist_seg).fillna(method="ffill").fillna(method="bfill").values)
        if np.isnan(fut_seg).any():
            fut_seg = (pd.Series(fut_seg).fillna(method="ffill").fillna(method="bfill").values)
        full_seg = np.concatenate([hist_seg, fut_seg])
        cand_full_segments.append(full_seg.astype(np.float32))
        cand_series.append(uid)
        cand_end_idx.append(end)
        cand_start_idx.append(start)

cand_full_segments = (np.stack(cand_full_segments)
                      if cand_full_segments else np.zeros((0, H+P), dtype=np.float32))

cand_series    = np.array(cand_series)
cand_end_idx   = np.array(cand_end_idx)
cand_start_idx = np.array(cand_start_idx)

print(f"[BUILD] Candidate segments: {cand_full_segments.shape} built in {time.time() - t_build:.2f}s")

uid_to_cand_indices = {}
for i, s in enumerate(cand_series):
    uid_to_cand_indices.setdefault(s, []).append(i)

# History matrix for MI scoring
cand_hist_matrix = cand_full_segments[:, :H] if cand_full_segments.shape[0] else np.zeros((0, H), dtype=np.float32)

# ======================================================================
# Helpers
# ======================================================================
def window_overlap_ratio(a_start, a_end, b_start, b_end):
    """
    Overlap length / window length. Windows are [start, end) in indices.
    Assumes equal lengths (H), but formula works generally.
    """
    inter = max(0, min(a_end, b_end) - max(a_start, b_start))
    length = max(a_end - a_start, 1)
    return inter / length

# ======================================================================
# MI-Inspired Retrieval
# ======================================================================
def mi_score_histories(query_hist: np.ndarray,
                       candidate_hist_matrix: np.ndarray,
                       n_neighbors: int = 3,
                       var_eps: float = 1e-8) -> np.ndarray:
    if candidate_hist_matrix.shape[0] == 0:
        return np.zeros((0,), dtype=float)

    X = candidate_hist_matrix.T 
    y = query_hist.astype(float)
    var_mask = X.var(axis=0) > var_eps
    if not var_mask.any():
        return np.zeros((candidate_hist_matrix.shape[0],), dtype=float)
    X_use = X[:, var_mask]
    scores_sub = mutual_info_regression(X_use, y, discrete_features=False, n_neighbors=n_neighbors)
    scores = np.zeros(X.shape[1], dtype=float)
    scores[var_mask] = scores_sub
    return scores

def mi_retrieve_topn(query_series, query_uid, cutoff_idx, top_n, n_neighbors=3):
    if top_n == 0:
        return np.array([], dtype=int), np.array([], dtype=float), query_series[cutoff_idx - H:cutoff_idx]
    if cutoff_idx is None or cutoff_idx < H or cutoff_idx + P > len(query_series):
        return None
    ctx = query_series[cutoff_idx - H:cutoff_idx]
    if len(ctx) != H or np.isnan(ctx).all():
        return None
    if np.isnan(ctx).any():
        ctx = (pd.Series(ctx).fillna(method="ffill").fillna(method="bfill")).values
    if cand_hist_matrix.shape[0] == 0:
        return np.array([], dtype=int), np.array([], dtype=float), ctx

    scores = mi_score_histories(ctx, cand_hist_matrix)

    if query_uid in uid_to_cand_indices:
        for idx in uid_to_cand_indices[query_uid]:
            if cand_end_idx[idx] >= (cutoff_idx - 1):
                scores[idx] = -np.inf

    valid = ~np.isneginf(scores)
    if not valid.any():
        return np.array([], dtype=int), np.array([], dtype=float), ctx

    vidx = np.where(valid)[0]
    vscores = scores[valid]

    # sort by MI desc
    order = np.argsort(vscores)[::-1]

    chosen_list = []
    score_list  = []

    for pos in order:
        if len(chosen_list) >= top_n:
            break
        cand_i = int(vidx[pos])
        keep = True
        for taken in chosen_list:
            if cand_series[cand_i] == cand_series[taken]:
                ov = window_overlap_ratio(
                    cand_start_idx[cand_i], cand_end_idx[cand_i],
                    cand_start_idx[taken],   cand_end_idx[taken]
                )
                if ov >= OVERLAP_THRESH:
                    keep = False
                    break
        if keep:
            chosen_list.append(cand_i)
            score_list.append(float(vscores[pos]))

    return np.array(chosen_list, dtype=int), np.array(score_list, dtype=float), ctx

# ======================================================================
# Augmentation
# ======================================================================
def build_augmented_variants(query_ctx, top_indices):
    ctx_t = torch.tensor(query_ctx, dtype=torch.float32)
    m_c = float(ctx_t.mean())
    s_c = float(ctx_t.std(unbiased=False) + 1e-7)
    ctx_n = (ctx_t - m_c) / s_c

    if len(top_indices) == 0:
        return [ctx_n], [(m_c, s_c)], True
    
    variants = []
    stats = []
    zero_prefix = torch.zeros(H + P, dtype=torch.float32)
    anchor_variant = torch.cat([zero_prefix, ctx_n], dim=0)
    variants.append(anchor_variant)
    stats.append((m_c, s_c))

    for ci in top_indices:
        seg_full = cand_full_segments[ci]
        seg_t = torch.tensor(seg_full, dtype=torch.float32)
        mask_s = ~torch.isnan(seg_t)
        if mask_s.any():
            m_s = float(seg_t[mask_s].mean())
            s_s = float(seg_t[mask_s].std(unbiased=False) + 1e-7)
        else:
            m_s, s_s = 0.0, 1.0
        seg_n = (seg_t - m_s) / s_s
        shift = ctx_n[0] - seg_n[H - 1]
        seg_n = seg_n + shift
        combined = torch.cat([seg_n, ctx_n], dim=0)
        variants.append(combined)
        stats.append((m_c, s_c))

    for v in variants:
        assert v.shape[0] == AUG_LEN, f"Variant length mismatch {v.shape[0]} != {AUG_LEN}"
    return variants, stats, False

# ======================================================================
# Forecast loop
# ======================================================================
retrieval_audit = []
records = []
eval_dates = pd.date_range(eval_start, eval_end, freq="D")

t_loop = time.time()
for ds in tqdm(eval_dates, desc="MI Style Retrieval"):
    for uid in eval_uids:
        cutoff = date_to_cutoff(uid, ds)
        if cutoff is None or cutoff < min_history_points or cutoff + P > len(series_map[uid]):
            continue
        series_vals = np.asarray(series_map[uid], dtype=float)

        ret = mi_retrieve_topn(series_vals, uid, cutoff, top_n)
        if ret is None:
            continue
        top_idx, top_scores, query_ctx = ret

        augmented_variants, stats, baseline_mode = build_augmented_variants(query_ctx, top_idx)

        with torch.no_grad():
            samples = chronos.predict(
                augmented_variants,
                prediction_length=P,
                num_samples=chronos.faux_num_samples
            )
        fc_matrix = samples.mean(dim=1).cpu().numpy()
        K = fc_matrix.shape[0]

        fc_matrix_denorm = np.zeros_like(fc_matrix)
        for i in range(K):
            m_c, s_c = stats[i]
            fc_matrix_denorm[i] = fc_matrix[i] * s_c + m_c
            
        if baseline_mode:
            final_fc = fc_matrix_denorm[0]
            progressive = [fc_matrix_denorm[0]]
        else:
            if aggregate_mode == "mean":
                final_fc = fc_matrix_denorm.mean(axis=0)
                progressive = [fc_matrix_denorm[:i].mean(axis=0) for i in range(1, K + 1)]
            elif aggregate_mode == "dist_weighted":
                retrieved_scores = top_scores
                dists = (retrieved_scores.max() - retrieved_scores) + 1e-8
                anchor_proxy = np.median(dists) if dists.size else 1.0
                d_all = np.concatenate([[anchor_proxy], dists])
                w_all = np.exp(-alpha * d_all); w_all /= w_all.sum()
                final_fc = (fc_matrix_denorm * w_all[:, None]).sum(axis=0)
                progressive = []
                for m in range(1, K + 1):
                    d_part = d_all[:m]
                    w_part = np.exp(-alpha * d_part); w_part /= w_part.sum()
                    progressive.append((fc_matrix_denorm[:m] * w_part[:, None]).sum(axis=0))
            elif aggregate_mode == "mi_weighted":
                anchor_score = np.median(top_scores) if top_scores.size else 1.0
                all_scores = np.concatenate([[anchor_score], top_scores])
                w_all = all_scores / all_scores.sum()
                final_fc = (fc_matrix_denorm * w_all[:, None]).sum(axis=0)
                progressive = []
                for m in range(1, K + 1):
                    w_part = all_scores[:m]
                    w_part = w_part / w_part.sum()
                    progressive.append((fc_matrix_denorm[:m] * w_part[:, None]).sum(axis=0))
            else:
                raise ValueError("Unsupported aggregate_mode")

        truth_future = series_vals[cutoff:cutoff + P]
        mae_anchor = float(np.mean(np.abs(fc_matrix_denorm[0] - truth_future)))
        mae_final  = float(np.mean(np.abs(final_fc - truth_future)))

        audit_rec = {
            "unique_id": uid,
            "ds": str(ds),
            "cutoff_index": int(cutoff),
            "baseline_mode": baseline_mode,
            "num_retrieved": 0 if baseline_mode else int(len(top_idx)),
            "anchor_future_mae": mae_anchor,
            "ensemble_future_mae": mae_final,
            "aggregation": aggregate_mode,
            "retrieved": [],
            "progressive_mae": [
                {
                    "num_contexts": i,
                    "mae": float(np.mean(np.abs(progressive[i-1] - truth_future)))
                } for i in range(1, len(progressive) + 1)
            ]
        }

        if not baseline_mode:
            for rank, (ci, score) in enumerate(zip(top_idx, top_scores), 1):
                raw_seg = cand_full_segments[ci]
                fut_seg = raw_seg[H:H+P]
                start_idx = cand_start_idx[ci]
                end_idx   = cand_end_idx[ci]
                audit_rec["retrieved"].append({
                    "rank": rank,
                    "mi_score": float(score),
                    "source_uid": cand_series[ci],
                    "candidate_index": int(ci),
                    "window_start_index": int(start_idx),
                    "window_end_index": int(end_idx),
                    "window_start_date": str(dates_map[cand_series[ci]][start_idx]),
                    "window_end_date": str(dates_map[cand_series[ci]][end_idx - 1]),
                    "future_values": fut_seg.tolist()
                })

        retrieval_audit.append(audit_rec)

        # store rows
        for h in range(P):
            records.append({
                "unique_id": uid,
                "ds": ds,
                "h": h + 1,
                "pred": float(final_fc[h]),
                "truth": float(truth_future[h])
            })

print(f"[LOOP] Done in {time.time() - t_loop:.2f}s; audit entries: {len(retrieval_audit)}")

# Define output path for raw predictions CSV
raw_preds_path = Path(f"KSTUDY_raw_preds_top{top_n}_{prediction_length}days_MI.csv")

# ----------------------------------------------------------------------
# METRICS  (overall, per-station) and write raw preds CSV
# ----------------------------------------------------------------------
df_aug = pd.DataFrame(records)
if df_aug.empty:
    raise RuntimeError("No rows produced. Adjust config/date ranges/data availability.")

# Write raw preds & truth to a single CSV
df_aug.to_csv(raw_preds_path, index=False)
logger.info(f"Wrote raw predictions to {raw_preds_path}")

# Overall metrics across all points
err_all     = df_aug.pred - df_aug.truth
overall_mae = float(np.mean(np.abs(err_all)))
overall_rmse= float(np.sqrt(np.mean(err_all**2)))

print("=== MI Style Retrieval ===")

# Per-station metrics
def station_metrics(group):
    err  = group.pred - group.truth
    mae  = float(np.mean(np.abs(err)))
    rmse = float(np.sqrt(np.mean(err**2)))
    return pd.Series({"MAE": mae, "RMSE": rmse})

per_station = (
    df_aug
      .groupby("unique_id")
      .apply(station_metrics)
      .reset_index()
      .rename(columns={"unique_id": "station"})
)

print("Per-station metrics:")
print(per_station.to_string(index=False))

# Average across stations
avg_mae  = per_station.MAE.mean()
avg_rmse = per_station.RMSE.mean()
print(f"\nAVERAGE ACROSS STATIONS: MAE={avg_mae:.6f} | RMSE={avg_rmse:.6f}")